In [ ]:
# Importation des bibliothèques et librairies Python
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# Définir les chemins des dossiers contenant les images suspectes et non suspectes
path_suspect = 'https://drive.google.com/drive/folders/1AMZgATADZ7OPAbn2Y0vtTFrruTvOS3Zx'
path_non_suspect = 'https://drive.google.com/drive/folders/1ihkK0SdPtkRgW4t4R-_HMZALUBNxjA4G'

In [ ]:
# Créer des générateurs d'images pour les données d'entraînement et de test
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    directory=path_suspect,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    directory=path_suspect,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Charger le modèle pré-entraîné MobileNetV2
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Ajouter des couches supplémentaires pour la classification binaire
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraîner le modèle
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
# Prédire les classes pour les données de test
y_pred = model.predict(val_generator)
y_pred = y_pred > 0.5

In [ ]:
# Calculer la matrice de confusion
cm = confusion_matrix(val_generator.classes, y_pred)

In [ ]:
# Afficher la matrice de confusion
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Matrice de confusion')
plt.colorbar()
plt.xlabel('Vraies classes')
plt.ylabel('Classes prédites')
plt.show()

In [ ]:
# Calculer la courbe ROC
fpr, tpr, thresholds = roc_curve(val_generator.classes, y_pred)
roc_auc = roc_auc_score(val_generator.classes, y_pred)

In [ ]:
# Afficher la courbe ROC
plt.plot(fpr, tpr, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()